In [2]:
#carico i dati del documentale (amail_sales_new.csv)
from google.colab import files
files.upload()

Saving email_sales_new.csv to email_sales_new.csv


{'email_sales_new.csv': b'title;messaggio_all\r\n"Variazione;Amministrativo;Anagrafica"; Il cliente id215040 ha due contratti pib, quello con id906154 ha intestazione e sede legale errata. Allego richiesta per correggere fatture e inoltre ho aggiunto la mail PEC. cambio intestazione e sede BARACCHI id215040\r\n"Variazione;Amministrativo;Anagrafica"; Buon pomeriggio,il cliente riferisce di non ricevere fatture all indirizzo mailmindicato.Peetanto vi chiedo di verificare.L indirizzo corretto e: c.donofrio1985@gmail.com Grazie. Modifica indirizzo mail d onofrio carmine 319919\r\n"Variazione;Amministrativo;Anagrafica"; Ciao il cliente mi dice che per Atripalda il civico corretto \xe8 6 non 8.Se cortesemente puoi provvedere a rettificarlo nella intestazione fattura.Per Avellino tutto ok.Grazie e buon lavoro Luigi Pepe Cel.3420223635 R: Re: Id.308106\r\n"Variazione;Amministrativo;Anagrafica"; Buon Giorno Il cliente in oggetto chiede che venga cambiata la ragione sociale della fornitura luce 

In [3]:
#clono il repository di bert da github
!git clone https://github.com/google-research/bert.git

Cloning into 'bert'...
remote: Enumerating objects: 336, done.
remote: Total 336 (delta 0), reused 0 (delta 0), pack-reused 336
Receiving objects: 100% (336/336), 291.40 KiB | 1.46 MiB/s, done.
Resolving deltas: 100% (184/184), done.


In [4]:
#importo il mio run_classifier
from google.colab import files
files.upload()

Saving run_classifier.py to run_classifier.py


{'run_classifier.py': b'# coding=utf-8\n# Copyright 2018 The Google AI Language Team Authors.\n#\n# Licensed under the Apache License, Version 2.0 (the "License");\n# you may not use this file except in compliance with the License.\n# You may obtain a copy of the License at\n#\n#     http://www.apache.org/licenses/LICENSE-2.0\n#\n# Unless required by applicable law or agreed to in writing, software\n# distributed under the License is distributed on an "AS IS" BASIS,\n# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.\n# See the License for the specific language governing permissions and\n# limitations under the License.\n"""BERT finetuning runner."""\n\nfrom __future__ import absolute_import\nfrom __future__ import division\nfrom __future__ import print_function\n\nimport collections\nimport csv\nimport os\nimport modeling\nimport optimization\nimport tokenization\nimport tensorflow as tf\nimport sys\n\nflags = tf.flags\n\nFLAGS = flags.FLAGS\n\n## Required para

In [0]:
#rimuovo il vecchio run_classifier
!rm -r ./bert/run_classifier.py

In [0]:
#sposto nella cartella di bert il mio run_classifier
!mv ./run_classifier.py ./bert/run_classifier.py

In [7]:
#scarico bert pre-trained
!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
!unzip the file
!unzip multi_cased_L-12_H-768_A-12.zip

--2020-01-02 11:39:43--  https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.128, 2607:f8b0:400e:c07::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 662903077 (632M) [application/zip]
Saving to: ‘multi_cased_L-12_H-768_A-12.zip’

multi_cased_L-12_H- 100%[===================>] 632.19M   116MB/s    in 5.8s    

2020-01-02 11:39:49 (109 MB/s) - ‘multi_cased_L-12_H-768_A-12.zip’ saved [662903077/662903077]

unzip:  cannot find or open the, the.zip or the.ZIP.
Archive:  multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_m

In [8]:
!pip3 install virtualenv
!virtualenv bertenv

     |████████████████████████████████| 3.4MB 4.6MB/s 
Using base prefix '/usr'
New python executable in /content/bertenv/bin/python3
Also creating executable in /content/bertenv/bin/python
Installing setuptools, pip, wheel...
done.


In [0]:
!source ./bertenv/bin/activate

In [0]:
import os
os.mkdir("bert_output")
os.mkdir("data")

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
from pandas import DataFrame
import numpy as np

df = pd.read_csv("./email_sales_new.csv", delimiter = ';',encoding="latin_1")

In [0]:
df=df.dropna()

df_bert = pd.DataFrame({'user_id': range(1, len(df) + 1),
  'label': df['title'],
  'alpha': ['a']*df.shape[0],
  'text': df['messaggio_all'].replace(r'\n',' ',regex=True)})

from sklearn.preprocessing import LabelEncoder
labelencoder_X = LabelEncoder()
df_bert['one_hot_label'] = labelencoder_X.fit_transform(df_bert['label'])

df_bert_train, df_bert_test_completo = train_test_split(df_bert, test_size=0.2, random_state=1234,stratify=df_bert["label"])

df_bert_train =  pd.DataFrame({'user_id': df_bert_train['user_id'],
  'label': df_bert_train['one_hot_label'],
  'alpha': df_bert_train['alpha'],
  'text':df_bert_train['text']})

df_bert_train, df_bert_dev = train_test_split(df_bert_train, test_size=0.10, random_state=1234
                                              #,stratify=df_bert["label"]
                                              )



df_bert_test =  pd.DataFrame({'user_id': df_bert_test_completo['user_id'],
  'text': df_bert_test_completo['text']})



In [0]:

df_bert_train.to_csv('./data/train.tsv', sep='\t', index=False, header=False, encoding="utf-8")
df_bert_dev.to_csv('./data/dev.tsv', sep='\t', index=False, header=False, encoding="utf-8")
df_bert_test.to_csv('./data/test.tsv', sep='\t', index=False, header=True, encoding="utf-8")
df_bert_test_completo.to_csv('./data/test_completo.tsv', sep='\t', index=False, header=True, encoding="utf-8")



In [0]:
#df_bert_train.to_csv('data/train.tsv', sep='\t', index=False, header=False)
#df_bert_dev.to_csv('data/dev.tsv', sep='\t', index=False, header=False)
#df_bert_test.to_csv('data/test.tsv', sep='\t', index=False, header=True)

In [15]:
from IPython.display import display, HTML
js = ('<script>function ConnectButton(){ '
           'console.log("Connect pushed"); '
           'document.querySelector("#connect").click()} '
           'setInterval(ConnectButton,3000);</script>')
display(HTML(js))

In [16]:
!python /content/bert/run_classifier.py 17\
 --task_name=cola\
 --do_train=true\
 --do_eval=true\
 --data_dir=/content/data/\
 --vocab_file=/content/multi_cased_L-12_H-768_A-12/vocab.txt\
 --bert_config_file=/content/multi_cased_L-12_H-768_A-12/bert_config.json\
 --init_checkpoint=/content/multi_cased_L-12_H-768_A-12/bert_model.ckpt\
 --max_seq_length=128\
 --train_batch_size=32\
 --learning_rate=2e-5\
 --num_train_epochs=30.0\
 --output_dir=/content/bert_output/\
 --do_lower_case=False




W0102 11:40:11.517866 139798922905472 module_wrapper.py:139] From /content/bert/run_classifier.py:785: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0102 11:40:11.518015 139798922905472 module_wrapper.py:139] From /content/bert/run_classifier.py:785: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0102 11:40:11.518316 139798922905472 module_wrapper.py:139] From /content/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0102 11:40:11.518879 139798922905472 module_wrapper.py:139] From /content/bert/run_classifier.py:809: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  

In [18]:
#per inserire il test set nel modello
!python /content/bert/run_classifier.py 17 \
--task_name=cola \
--do_predict=true \
--data_dir=/content/data/ \
--vocab_file=/content/multi_cased_L-12_H-768_A-12/vocab.txt \
--bert_config_file=/content/multi_cased_L-12_H-768_A-12/bert_config.json \
--init_checkpoint=/content/bert_output/model.ckpt-1907 \
--max_seq_length=128 \
--output_dir=/content/bert_output/ \
--do_lower_case=False




W0102 13:33:28.375924 139774930478976 module_wrapper.py:139] From /content/bert/run_classifier.py:785: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W0102 13:33:28.376124 139774930478976 module_wrapper.py:139] From /content/bert/run_classifier.py:785: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W0102 13:33:28.376525 139774930478976 module_wrapper.py:139] From /content/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W0102 13:33:28.377103 139774930478976 module_wrapper.py:139] From /content/bert/run_classifier.py:809: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  

In [19]:
!ls ./bert_output

checkpoint
eval
eval_results.txt
eval.tf_record
events.out.tfevents.1577965223.8f2153d54375
graph.pbtxt
model.ckpt-0.data-00000-of-00001
model.ckpt-0.index
model.ckpt-0.meta
model.ckpt-1000.data-00000-of-00001
model.ckpt-1000.index
model.ckpt-1000.meta
model.ckpt-1907.data-00000-of-00001
model.ckpt-1907.index
model.ckpt-1907.meta
predict.tf_record
test_results.tsv
train.tf_record


In [0]:
#per calcolare l'accuracy sul test set
#read the results data for the probabilities
df_result = pd.read_csv('./bert_output/test_results.tsv', delimiter = '\t',encoding="utf-8",header=None)
df_test_completo = pd.read_csv('./data/test_completo.tsv', delimiter = '\t',encoding="utf-8")

In [21]:
df_test_completo.head()

,user_id,label,alpha,text,one_hot_label
0,158,Reclamo;Tecnico;Guasto,a,"Salve,Il cliente in questione lamenta una con...",9
1,1607,Informazione;Amministrativo;Subentro,a,"Ciao ragazzi, il cliente con id riportato in ...",3
2,1302,Informazione;Amministrativo;Richiesta dilazion...,a,Ciao Per il cliente in oggetto chiedo cortese...,1
3,704,Variazione;Amministrativo;Modalità di Pagamento,a,Allego modulo SDD per il cliente con codice i...,12
4,551,Variazione;Tecnico;MNP IN Post Attivazione,a,"Ciao ragazzi, Invio modulo mnp del cliente ri...",15


In [0]:
risultati_test = pd.DataFrame({'guid': df_test_completo['user_id'],
'text': df_test_completo['text'],
'label':df_test_completo['label'],
'one_hot_label':df_test_completo['one_hot_label'],
'predict_label': df_result.idxmax(axis=1),
'Prob' : df_result.max(axis=1)})

In [23]:
risultati_test.iloc[4,1]

' Ciao ragazzi, Invio modulo mnp del cliente riportato in oggetto Resto in attesa di gentile riscontro modulo mnp piccino anna id 349553'

In [0]:
x = pd.crosstab(risultati_test['one_hot_label'],risultati_test['predict_label'])

In [25]:
np.diag(x).sum()/risultati_test.shape[0]

0.9098939929328622

# **creazione tool per la predizione di frasi nuove**


In [0]:
os.mkdir('./Shell')

In [26]:
!sh ./percolab.sh "Salve ragazzi, il cliente 123456 dice di aver pagato più di quanto ha consumato, in allegato vi comunico l'autolettura. Grazie mille e buon lavoro. Soggetto: Comunicazione Autolettura. Da: Mario Rossi, Sales area Milano"

sh: 0: Can't open ./percolab.sh


In [0]:
!sh ./percolab.sh "Ciao ragazzi, il cliente non ha ancora ricevuto il modem, si possono avere notizie dal corriere?"




W1230 16:44:34.155909 139790987552640 module_wrapper.py:139] From /content/bert/run_classifier.py:785: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1230 16:44:34.156109 139790987552640 module_wrapper.py:139] From /content/bert/run_classifier.py:785: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1230 16:44:34.156552 139790987552640 module_wrapper.py:139] From /content/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1230 16:44:34.157252 139790987552640 module_wrapper.py:139] From /content/bert/run_classifier.py:809: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  

In [0]:
!sh ./percolab.sh "ciao, il cliente richiede rimodulazione dell'offerta"




W1230 15:50:41.381467 140082618292096 module_wrapper.py:139] From /content/bert/run_classifier.py:785: The name tf.logging.set_verbosity is deprecated. Please use tf.compat.v1.logging.set_verbosity instead.


W1230 15:50:41.381711 140082618292096 module_wrapper.py:139] From /content/bert/run_classifier.py:785: The name tf.logging.INFO is deprecated. Please use tf.compat.v1.logging.INFO instead.


W1230 15:50:41.382190 140082618292096 module_wrapper.py:139] From /content/bert/modeling.py:93: The name tf.gfile.GFile is deprecated. Please use tf.io.gfile.GFile instead.


W1230 15:50:41.383003 140082618292096 module_wrapper.py:139] From /content/bert/run_classifier.py:809: The name tf.gfile.MakeDirs is deprecated. Please use tf.io.gfile.makedirs instead.

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  

In [0]:
!zip -d ./bert_output/model.ckpt-1907.data-00000-of-00001

	zip warning: missing end signature--probably not a zip file (did you
	zip warning: remember to use binary mode when you transferred it?)
	zip warning: (if you are trying to read a damaged archive try -F)

zip error: Zip file structure invalid (./bert_output/model.ckpt-1907.data-00000-of-00001)


In [0]:
sh /home/utente/bert-master/dataset/documentale/Scrivi_newVersion.sh